In [6]:
!pip install pycountry

import pandas as pd
import pycountry
import sqlite3

# Create database
conn = sqlite3.connect(':memory:')
df.to_sql('repay_data', conn, index=False, if_exists='replace')
cursor = conn.cursor()


In [3]:
#Load Sample Data
file_path = '/content/ibrd_and_ida_net_flows_commitments_06-02-2025.csv'
df = pd.read_csv(file_path)

# Map 1 of 2: Reconfigure Column Headers to Match REPAY Core Naming Conventions
customRename = {
    'Financier': 'financier',
    'Fiscal Year': 'fiscalYear',
    'Region': 'region',
    'Country / Economy': 'countryName',
    'Gross Disbursement (US$)': 'grossDisbursement',
    'Repayments (US$)': 'repayments',
    'Net Disbursement (US$)': 'netDisbursement',
    'Interest (US$)': 'interest',
    'Fees (US$)': 'fees',
    'IBRD Commitments (US$)': 'ibrdCommitments',
    'IDA Grant Commitments (US$)': 'grantCommitments',
    'IDA Non-Concessional Commitments (US$)': 'nonconcessionalCommitments',
    'IDA Concessional Commitments (US$)': 'concessionalCommitments'
}

df.rename(columns=customRename, inplace=True)


# Map 2 of 2: Add 2-letter country code to each country name
def getCountry(name):
    try:
        return pycountry.countries.lookup(name).alpha_2
    except:
        return None

df['countryCode'] = df['countryName'].apply(getCountry)

# Append Country code (e.g. "France (FR)")
df['countryName'] = df.apply(
    lambda row: f"{row['countryName']} ({row['countryCode']})" if row['countryCode'] else row['countryName'],
    axis=1
)
df.drop(columns='countryCode', inplace=True)

In [5]:
# Export Clean File
output_path = '/content/Sample_remapped.csv'
df.to_csv(output_path, index=False)

# Preview 1 of 2: Original Dataset Snapshot (Before Migration Logic)
print("Preview 1 of 2: ORIGINAL File (Before Column Renaming & Country Code Mapping)")
original_df = pd.read_csv(file_path)
print(original_df.head())

Preview 1 of 2: ORIGINAL File (Before Column Renaming & Country Code Mapping)
  Financier  Fiscal Year                 Region Country / Economy  \
0      IBRD         2025  EAST ASIA AND PACIFIC             China   
1      IBRD         2025  EAST ASIA AND PACIFIC              Fiji   
2      IBRD         2025  EAST ASIA AND PACIFIC         Indonesia   
3      IBRD         2025  EAST ASIA AND PACIFIC          Mongolia   
4      IBRD         2025  EAST ASIA AND PACIFIC  Papua New Guinea   

   Gross Disbursement (US$)  Repayments (US$)  Net Disbursement (US$)  \
0              5.698201e+08      1.065798e+09           -4.959784e+08   
1              6.828037e+06      4.314512e+06            2.513525e+06   
2              5.499439e+08      1.190085e+09           -6.401414e+08   
3              1.039759e+07      1.076000e+06            9.321586e+06   
4              4.886066e+05      0.000000e+00            4.886066e+05   

   Interest (US$)  Fees (US$)  IBRD Commitments (US$)  \
0    7.5193

In [28]:
# Preview 2 of 2: Cleaned Dataset Snapshot (After Migration Logic)
print("\n Preview 2 of 2: CLEANED File (After Applying Migration Logic)")
print(df.head())


 Preview 2 of 2: CLEANED File (After Applying Migration Logic)
  financier  fiscalYear                 region            countryName  \
0      IBRD        2025  EAST ASIA AND PACIFIC             China (CN)   
1      IBRD        2025  EAST ASIA AND PACIFIC              Fiji (FJ)   
2      IBRD        2025  EAST ASIA AND PACIFIC         Indonesia (ID)   
3      IBRD        2025  EAST ASIA AND PACIFIC          Mongolia (MN)   
4      IBRD        2025  EAST ASIA AND PACIFIC  Papua New Guinea (PG)   

   grossDisbursement    repayments  netDisbursement      interest        fees  \
0       5.698201e+08  1.065798e+09    -4.959784e+08  7.519396e+08  4224725.14   
1       6.828037e+06  4.314512e+06     2.513525e+06  8.565158e+06        0.00   
2       5.499439e+08  1.190085e+09    -6.401414e+08  1.058008e+09  2828926.60   
3       1.039759e+07  1.076000e+06     9.321586e+06  3.134798e+06        0.00   
4       4.886066e+05  0.000000e+00     4.886066e+05  4.027383e+05        0.00   

   ibrdCom

In [7]:
#Python Query

import pandas as pd
import random

original_path = '/content/ibrd_and_ida_net_flows_commitments_06-02-2025.csv'
remapped_path = '/content/Sample_remapped.csv'

original_df = pd.read_csv(original_path)
remapped_df = pd.read_csv(remapped_path)

# Compare 5 random rows to validate successful transformation
print("Verifying Country Name Migration\n")

indices = random.sample(range(min(len(original_df), len(remapped_df))), 5)

for idx in indices:
    original_val = original_df.iloc[idx]['Country / Economy']
    cleaned_val = remapped_df.iloc[idx]['countryName']
    print(f"Row {idx + 1}:\n  Original → {original_val}\n  Cleaned  → {cleaned_val}\n")

print("\nField Check: Gross Disbursement\n")

for idx in indices:
    original_val = original_df.iloc[idx]['Gross Disbursement (US$)']
    cleaned_val = remapped_df.iloc[idx]['grossDisbursement']
    print(f"Row {idx + 1}:\n  Original → {original_val}\n  Cleaned  → {cleaned_val}\n")

print("Field Check: Repayments\n")

for idx in indices:
    original_val = original_df.iloc[idx]['Repayments (US$)']
    cleaned_val = remapped_df.iloc[idx]['repayments']
    print(f"Row {idx + 1}:\n  Original → {original_val}\n  Cleaned  → {cleaned_val}\n")

print("Field Check: Fiscal Year\n")

for idx in indices:
    original_val = original_df.iloc[idx]['Fiscal Year']
    cleaned_val = remapped_df.iloc[idx]['fiscalYear']
    print(f"Row {idx + 1}:\n  Original → {original_val}\n  Cleaned  → {cleaned_val}\n")

print("Field Check: Interest\n")

for idx in indices:
    original_val = original_df.iloc[idx]['Interest (US$)']
    cleaned_val = remapped_df.iloc[idx]['interest']
    print(f"Row {idx + 1}:\n  Original → {original_val}\n  Cleaned  → {cleaned_val}\n")

print("Field Check: Fees\n")

for idx in indices:
    original_val = original_df.iloc[idx]['Fees (US$)']
    cleaned_val = remapped_df.iloc[idx]['fees']
    print(f"Row {idx + 1}:\n  Original → {original_val}\n  Cleaned  → {cleaned_val}\n")



Verifying Country Name Migration

Row 204:
  Original → Eswatini
  Cleaned  → Eswatini (SZ)

Row 1820:
  Original → Tuvalu
  Cleaned  → Tuvalu (TV)

Row 2572:
  Original → Samoa
  Cleaned  → Samoa (WS)

Row 568:
  Original → Cote d'Ivoire
  Cleaned  → Cote d'Ivoire

Row 1506:
  Original → Mexico
  Cleaned  → Mexico (MX)


Field Check: Gross Disbursement

Row 204:
  Original → 29495850.31
  Cleaned  → 29495850.31

Row 1820:
  Original → 9412497.14
  Cleaned  → 9412497.14

Row 2572:
  Original → 3568538.01
  Cleaned  → 3568538.01

Row 568:
  Original → 959416579.8
  Cleaned  → 959416579.8

Row 1506:
  Original → 423304743.3
  Cleaned  → 423304743.3

Field Check: Repayments

Row 204:
  Original → 7222638.08
  Cleaned  → 7222638.08

Row 1820:
  Original → 0.0
  Cleaned  → 0.0

Row 2572:
  Original → 2266680.61
  Cleaned  → 2266680.61

Row 568:
  Original → 35365401.33
  Cleaned  → 35365401.33

Row 1506:
  Original → 875213893.0
  Cleaned  → 875213893.0

Field Check: Fiscal Year

Row 204:
 

In [8]:
#SQL Query
!pip install -q pandasql

import pandas as pd
import pandasql as psql

original_path = '/content/ibrd_and_ida_net_flows_commitments_06-02-2025.csv'
remapped_path = '/content/Sample_remapped.csv'

original_df = pd.read_csv(original_path)
remapped_df = pd.read_csv(remapped_path)

pysqldf = lambda q: psql.sqldf(q, globals())

# Record counts match
print("Record Count Check")
print(pysqldf("SELECT 'original' AS source, COUNT(*) AS total FROM original_df UNION ALL SELECT 'remapped', COUNT(*) FROM remapped_df"))

# Check for nulls
print("\nNull Check")
print(pysqldf("""
SELECT
  SUM(CASE WHEN countryName IS NULL THEN 1 ELSE 0 END) AS null_country,
  SUM(CASE WHEN region IS NULL THEN 1 ELSE 0 END) AS null_region,
  SUM(CASE WHEN fiscalYear IS NULL THEN 1 ELSE 0 END) AS null_fiscalYear
FROM remapped_df
"""))

# Sample spot check (5 random rows)
import random
indices = random.sample(range(min(len(original_df), len(remapped_df))), 5)
print("\n Spot Checks")
for idx in indices:
    orig = original_df.iloc[idx]['Country / Economy']
    new = remapped_df.iloc[idx]['countryName']
    print(f"Row {idx+1}: Original → {orig} | Remapped → {new}")

# Region grouping summary
print("\n Grouping Check")
print(pysqldf("""
SELECT region, COUNT(*) AS count
FROM remapped_df
GROUP BY region
ORDER BY count DESC
"""))


Record Count Check
     source  total
0  original   3079
1  remapped   3079

Null Check
   null_country  null_region  null_fiscalYear
0             0            0                0

 Spot Checks
Row 795: Original → Cameroon | Remapped → Cameroon (CM)
Row 2763: Original → Philippines | Remapped → Philippines (PH)
Row 2639: Original → Ecuador | Remapped → Ecuador (EC)
Row 1424: Original → Papua New Guinea | Remapped → Papua New Guinea (PG)
Row 2656: Original → St. Kitts and Nevis | Remapped → St. Kitts and Nevis

 Grouping Check
                         region  count
0   LATIN AMERICA AND CARIBBEAN    640
1       EUROPE AND CENTRAL ASIA    615
2   EASTERN AND SOUTHERN AFRICA    459
3    WESTERN AND CENTRAL AFRICA    456
4         EAST ASIA AND PACIFIC    447
5  MIDDLE EAST AND NORTH AFRICA    247
6                    SOUTH ASIA    180
7                         OTHER     23
8                        AFRICA     12
